In [11]:
import logging
import os
import sys
import shutil
import tempfile

import matplotlib.pyplot as plt
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)

pin_memory= torch.cuda.is_available()
device= torch.device("cuda"if torch.cuda.is_available()else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

MONAI version: 1.3.0
Numpy version: 1.26.1
Pytorch version: 2.1.0+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: /home/<username>/enviroments/mri-classifier/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: 0.22.0
scipy version: 1.11.3
Pillow version: 10.0.1
Tensorboard version: 2.14.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.16.0+cu121
tqdm version: 4.66.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.6
pandas version: NOT INSTALLED or UNKNOWN VERSION.
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml 

In [12]:
#set data directory
parent_directory = r"data/ADNIDenoise"

In [13]:
#create filepaths and labels
dir_labels = ["AD", "MCI", "CN"]
images = []
labels = []

label_count = 0

for i in dir_labels:
    dir_path = os.path.join(parent_directory, i)
    file_names = os.listdir(dir_path)
    for f in file_names:
        images.append(os.path.join(dir_path, f))
    for j in range(0, len(file_names)):
        labels.append(label_count)
    label_count += 1


In [15]:
# Define transforms
train_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((96, 96, 96)), RandRotate90()])
val_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((96, 96, 96))])

#divide train, test and validate
train_paths, temp_paths, train_labels, temp_labels = train_test_split(images, labels, test_size=0.4, random_state=42)
valid_paths, test_paths, valid_labels, test_labels = train_test_split(temp_paths, temp_labels, test_size=0.5, random_state=42)

In [16]:
# create a training data loader
train_ds = ImageDataset(image_files=train_paths, labels=train_labels, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=2, pin_memory=pin_memory)

# create a validation data loader
val_ds = ImageDataset(image_files=valid_paths, labels=valid_labels, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=2, num_workers=2, pin_memory=pin_memory)

In [17]:
model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels = 3).to(device)
checkpoint = torch.load("best_metric_model_classification3d_array.pth")
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [18]:
# Create DenseNet121, CrossEntropyLoss and Adam optimizer
loss_function = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), 1e-4)

# start a typical PyTorch training
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter()
max_epochs = 20

for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0

    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()

        num_correct = 0.0
        metric_count = 0
        for val_data in val_loader:
            val_images, val_labels = val_data[0].to(device), val_data[1].to(device)
            with torch.no_grad():
                val_outputs = model(val_images)
                value = torch.eq(val_outputs.argmax(dim=1), val_labels)
                metric_count += len(value)
                num_correct += value.sum().item()

        metric = num_correct / metric_count
        metric_values.append(metric)

        if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), "best_metric_model_classification3d_array.pth")
            print("saved new best metric model")

        print(f"Current epoch: {epoch+1} current accuracy: {metric:.4f} ")
        print(f"Best accuracy: {best_metric:.4f} at epoch {best_metric_epoch}")
        writer.add_scalar("val_accuracy", metric, epoch + 1)

print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
writer.close()

----------
epoch 1/20
1/484, train_loss: 0.0021
2/484, train_loss: 0.0020
3/484, train_loss: 0.0038
4/484, train_loss: 0.0073
5/484, train_loss: 0.0011
6/484, train_loss: 0.0030
7/484, train_loss: 0.0083
8/484, train_loss: 0.0018
9/484, train_loss: 0.0011
10/484, train_loss: 0.0086
11/484, train_loss: 0.0036
12/484, train_loss: 0.0035
13/484, train_loss: 0.3569
14/484, train_loss: 0.0051
15/484, train_loss: 0.4789
16/484, train_loss: 0.0013
17/484, train_loss: 0.0039
18/484, train_loss: 0.0028
19/484, train_loss: 0.0016
20/484, train_loss: 0.1176
21/484, train_loss: 0.0031
22/484, train_loss: 0.0158
23/484, train_loss: 0.0697
24/484, train_loss: 0.0054
25/484, train_loss: 0.0327
26/484, train_loss: 0.0096
27/484, train_loss: 0.0260
28/484, train_loss: 0.0044
29/484, train_loss: 0.0017
30/484, train_loss: 0.5916
31/484, train_loss: 0.3266
32/484, train_loss: 0.0020
33/484, train_loss: 1.4066
34/484, train_loss: 0.2761
35/484, train_loss: 0.0030
36/484, train_loss: 0.0010
37/484, train_l

In [19]:
test_ds = ImageDataset(image_files=test_paths, labels=test_labels, transform=val_transforms)
test_loader = DataLoader(test_ds, batch_size=2, num_workers=2, pin_memory=pin_memory)

In [20]:
model.load_state_dict(torch.load("best_metric_model_classification3d_array.pth"))
model.eval()

DenseNet121(
  (features): Sequential(
    (conv0): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (norm0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (norm2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv3d(128, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
      )
      (denselayer2): _DenseLayer(
        (layers): Sequential(
 

In [21]:
predictions = []
ground_truth = []

for test_data in test_loader:
    test_images, test_labels = test_data[0].to(device), test_data[1].to(device)
    with torch.no_grad():
        test_outputs = model(test_images)
        # Perform any post-processing on the test_outputs if needed
        predictions.extend(test_outputs.cpu().numpy())  # Assuming you want to store the predictions

    ground_truth.extend(test_labels.cpu().numpy())

In [22]:
# Convert the list of predictions and ground truth to numpy arrays if not already done
predictions = np.array(predictions)
ground_truth = np.array(ground_truth)

accuracy = accuracy_score(ground_truth, predictions.argmax(axis=1))
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.8576
